# Run Main.py then Run next lines for Bokeh Interactive plot

In [8]:
%run main.py



reading from the following configuration files: 

  ['./config/data/pdf_files.yaml']




Reading in 54 file(s)






Running the following preprocessing actions:


{'add_stop_list': [], 'new_stop_set_list': [], 'SnowballStemmer': False, 'remove_stop_list': [], 'lemmatize': True, 'PorterStemmer': False, 'lowercase': True, 'use_stop_list': True, 'alpha_only': True}
doc_names

 ['/users/597667/scitext-explorer/test/data/BinLaden\\Addendum to the report of the Islamic Maghreb.pdf', '/users/597667/scitext-explorer/test/data/BinLaden\\Administrative instructions.pdf', '/users/597667/scitext-explorer/test/data/BinLaden\\AQ Accounting Ledger.pdf', '/users/597667/scitext-explorer/test/data/BinLaden\\Draft of a letter to subordinates.pdf', '/users/597667/scitext-explorer/test/data/BinLaden\\Duties of administrators.pdf', '/users/597667/scitext-explorer/test/data/BinLaden\\General instructions.pdf', '/users/597667/scitext-explorer/test/data/BinLaden\\In regard to the money that is in Sudan.pdf'

modeldum
n/n/n/
{2: {'Cluster Colors': ['#a6cee3', '#a6cee3', '#1f78b4', '#a6cee3', '#a6cee3', '#a6cee3', '#1f78b4', '#1f78b4', '#a6cee3', '#a6cee3', '#a6cee3', '#a6cee3', '#a6cee3', '#1f78b4', '#1f78b4', '#a6cee3', '#a6cee3', '#1f78b4', '#a6cee3', '#a6cee3', '#1f78b4', '#a6cee3', '#a6cee3', '#1f78b4', '#a6cee3', '#1f78b4', '#1f78b4', '#1f78b4', '#1f78b4', '#a6cee3', '#a6cee3', '#1f78b4', '#1f78b4', '#1f78b4', '#1f78b4', '#a6cee3', '#1f78b4', '#a6cee3', '#a6cee3', '#a6cee3', '#1f78b4', '#1f78b4', '#a6cee3', '#a6cee3', '#1f78b4', '#1f78b4', '#1f78b4', '#a6cee3', '#1f78b4', '#a6cee3', '#1f78b4', '#1f78b4', '#a6cee3', '#1f78b4'], 'KMeans Centroids': array([[ 0, 13, 30, 27, 16, 31, 14, 35, 22,  8, 49, 45, 23, 50,  7, 44,
        48, 37, 17, 40, 46,  9, 29, 32, 47, 18,  3, 43, 15, 53, 34, 28,
        24, 11, 51, 52, 42, 21, 33, 25, 26, 38, 36,  6,  5, 39, 41, 20,
        12, 10,  4, 19,  2,  1],
       [ 0,  1,  3, 12,  4,  6, 11, 19, 21, 10,  7, 41, 39,  5, 38, 20,
        25, 26, 42, 28, 

# Bokeh Imports

In [1]:
import pandas as pd

from bokeh.io import output_notebook, output_file, show, curdoc
from bokeh.plotting import figure
from bokeh.layouts import column, row, WidgetBox
from bokeh.models.widgets import CheckboxGroup, Slider, RangeSlider, Tabs, TextInput, Panel, RadioButtonGroup
from bokeh.models import CustomJS, ColumnDataSource, HoverTool, BoxSelectTool, CrosshairTool, SaveTool, Div, PanTool

output_notebook()
output_file('Bokeh_Test.html')

Loading BokehJS ...

# Load Dataframe

In [9]:
# filename = 'Bokeh Resume Test.xlsx'
filename = 'Bokeh_Test.xlsx'

df = pd.read_excel(filename)

# Create Dataset

In [5]:
cluster_sets = [str(i).zfill(2) for i in range(2,len(df.columns) - 1)]

dataset = dict()
for colname in df.columns:
#     print(colname)
    if colname in ['x','y','docname']:
        dataset[colname.replace(' ','')] = df[colname].tolist()
    else:
        dataset[str(colname)] = [[c[1:-2].split(", '")[0].strip(),c[1:-2].split(", '")[1].strip()] for c in df[colname].tolist()]
#         dataset[str(colname)] = [c[1:-2].split(", '")[1].strip() for c in resumes[colname].tolist()]
#         dataset['ClusterId'] = [c[1:-2].split(", '")[0].strip() for c in resumes[colname].tolist()]
        
## Starting Color (Gray)
dataset['color'] = ['lightgray'] * len(df)

## Starting size
starting_size = 15
dataset['size'] = [starting_size] * len(df)

## Starting size
dataset['ClusterId'] = [0] * len(df)

# Plot Bokeh

In [7]:
import random

cluster_sets = [str(i).zfill(2) for i in range(2,len(df.columns) - 1)]

source = ColumnDataSource(dataset)

## Configure the HoverTool popup dialog
hover = HoverTool(tooltips=[('Document Name', '@docname'),('Cluster Id','@ClusterId')])    

## Create a Bokeh figure object
p = figure(plot_width=600, plot_height=400, 
           tools = [hover,BoxSelectTool(),CrosshairTool(),SaveTool(),PanTool()], toolbar_location="above",
           title = 'Bokeh')

# add a circle renderer with a size, color, and alpha
p.circle('x', 'y', size = 'size', source = source,
         line_color = 'dimgray', line_width = 0.1,
         fill_color = 'color', fill_alpha = 0.6)

def size_callback(source=source, window=None):
    
    data = source.data
    new_size = cb_obj.value
    
    data['size'] = [new_size] * len(data['size'])
        
    source.change.emit()
        
def color_callback(source=source, window=None):
    
    print('ColorCALLBACK!!!')
    
    global color_dict
    
    data = source.data
    new_color = cb_obj.active
    
    data['color'] = [c[1] for c in data[str(new_color + 2)]]
    data['ClusterId'] = [c[0] for c in data[str(new_color + 2)]]
        
    source.change.emit()
    
## Slider to set size of dots
size_of_dots = Slider(start = 1, end = 100, step = 1, value = starting_size,
                        title = 'Scatter Plot Document Size', callback=CustomJS.from_py_func(size_callback))

## Radio Buttons to set color of dots
clusters = RadioButtonGroup(labels = cluster_sets, name = 'Number of Clusters', #active = 0,
                            callback=CustomJS.from_py_func(color_callback))

layout = column(size_of_dots, clusters, p)

show(layout)